In [1]:
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_

In [2]:
import re
import nltk
from nltk.corpus import shakespeare

nltk.download('shakespeare')


def extract_sentences_from_work(work):
    work_text = shakespeare.raw(work)
    sentences = []

    # Use regular expressions to find <SPEAKER> and <LINE> tags and extract the text
    speaker_pattern = re.compile(r'<SPEAKER>(.*?)</SPEAKER>', re.DOTALL)
    line_pattern = re.compile(r'<LINE>(.*?)</LINE>', re.DOTALL)

    speakers = speaker_pattern.findall(work_text)
    lines = line_pattern.findall(work_text)

    # Combine speaker and line text into sentences
    for speaker, line in zip(speakers, lines):
        # Remove any leading or trailing whitespace from the speaker and line
        speaker = speaker.strip()
        line = line.strip()
        # Combine speaker and line into a sentence
        sentence = f"{speaker}: {line}"
        sentences.append(sentence)

    return sentences


def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove characters that are not a-z or space
    cleaned_text = re.sub(r'[^a-z\s]', '', text)
    return cleaned_text


def validate_text(all_works_sentences):
    valid = True
    for work_sentences in all_works_sentences:
        for sentence in work_sentences:
            if not all(char in 'abcdefghijklmnopqrstuvwxyz ' for char in sentence):
                valid = False
                return valid
    return valid


def train_test_split(corpus):
    corpus_len = len(corpus)
    train_set_tmp = corpus[:corpus_len-1]
    train_set = []
    for work_lst in train_set_tmp:
        for paragraph in work_lst:
            train_set.append(paragraph)

    test_set_tmp = corpus[corpus_len-1:]
    test_set = []
    for work_lst in test_set_tmp:
        for paragraph in work_lst:
            test_set.append(paragraph)

    return train_set, test_set


def extract_unique_n_minus_1_grams(corpus, n):
    n_minus_1_grams = set()
    for sentence in corpus:
        sentence = ''.join(sentence)
        for i in range(len(sentence) - n + 2):
            context = sentence[i:i + n - 1]
            n_minus_1_grams.add(context)
    return n_minus_1_grams

def get_train_test_data():
    works = shakespeare.fileids()

    all_works_sentences = []
    for work in works:
        sentences = extract_sentences_from_work(work)
        cleaned_sentences = [list(clean_text(sentence)) for sentence in sentences]
        all_works_sentences.append(cleaned_sentences)

    if validate_text(all_works_sentences):
        print(f'Dictionary size is correct!')
    else:
        print(f'Dictionary size is wrong!')
    train_set, test_set = train_test_split(all_works_sentences)

    return all_works_sentences, train_set, test_set


[nltk_data] Downloading package shakespeare to /root/nltk_data...
[nltk_data]   Unzipping corpora/shakespeare.zip.


In [3]:
sents, train_set, test_set = get_train_test_data()

Dictionary size is correct!


In [5]:
from transformers import PreTrainedTokenizer, TFOpenAIGPTLMHeadModel
from transformers import OpenAIGPTLMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import OpenAIGPTConfig
from transformers import OpenAIGPTTokenizer
import torch
import tensorflow as tf
from nltk.corpus import brown
import re
import nltk
from collections import Counter
from datasets import Dataset

class CustomTokenizer(PreTrainedTokenizer):
    def __init__(self, vocab):
        self.vocab = vocab
        super().__init__(vocab=vocab, pad_token="<pad>", unk_token="<unk>")

        self.ids_to_tokens = {v: k for k, v in vocab.items()}
        self.pad_token = "<pad>"
        self.unk_token = "<unk>"
        self.pad_token_id = vocab[self.pad_token]

    def _tokenize(self, text):
        return list(text)

    def _convert_token_to_id(self, token):
        return self.vocab.get(token, self.vocab.get("<unk>"))

    def _convert_ids_to_tokens(self, index):
        return self.ids_to_tokens.get(index, "<unk>")

    def get_vocab(self):
        return self.vocab

    def encode(self, text, max_length=None, padding=True, truncation=True):
        tokens = self._tokenize(text)
        token_ids = [self._convert_token_to_id(token) for token in tokens]
        if truncation and max_length:
            token_ids = token_ids[:max_length]
        if padding and max_length:
            token_ids += [self._convert_token_to_id(self.pad_token)] * (max_length - len(token_ids))
        #attention_mask = [1] * len(token_ids)  # Add attention mask
        return token_ids

    def decode(self, token_ids):
        tokens = [self._convert_ids_to_tokens(token_id) for token_id in token_ids]
        return ''.join(tokens)

def prepare_data(corpus_tokenized):
    corpus_texts = [''.join(sentence) for sentence in corpus_tokenized]
    return Dataset.from_dict({'text': corpus_texts})



In [6]:
flat_corpus_t = [item for sublist in train_set for item in sublist]
flat_corpus_v = [item for sublist in test_set for item in sublist]
flat_corpus = flat_corpus_t + flat_corpus_v

vocab_counter = Counter(flat_corpus)
vocab = {word: idx for idx, (word, _) in enumerate(vocab_counter.items(), start=1)}
vocab["<pad>"] = 0  # Add padding token

train_text = "\n".join(["".join(sent) for sent in train_set])
test_text = "\n".join(["".join(sent) for sent in test_set])
with open('train.txt', 'w') as f:
    f.write(train_text)

with open('test.txt', 'w') as f:
    f.write(test_text)


In [7]:
from datasets import load_dataset

dataset = load_dataset('text', data_files={'train': 'train.txt', 'validation': 'test.txt'})

train_dataset = dataset['train']
validation_dataset = dataset['validation']


tokenizer = CustomTokenizer(vocab)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=False, max_length=256)
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Parameter 'function'=<function tokenize_function at 0x7909125ef130> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/6096 [00:00<?, ? examples/s]

Map:   0%|          | 0/841 [00:00<?, ? examples/s]

In [8]:
print(len(tokenized_train_dataset['input_ids'][4]))
#print(tokenized_train_dataset['input_ids'][1])
tokenizer._tokenize(train_dataset['text'][1])
print(train_dataset['text'][1])

49
cleopatra oerflows the measure those his goodly eyes


In [9]:
config = OpenAIGPTConfig.from_json_file("/content/drive/MyDrive/Colab Notebooks/shakespeare-scratch/config.json")
model = OpenAIGPTLMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/shakespeare-scratch/model.safetensors", config=config)


#config = OpenAIGPTConfig()
#model = OpenAIGPTLMHeadModel(config)
model.resize_token_embeddings(len(vocab))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(28, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (lm_head): Linear(in_features=768, out_features=28, bias=False)
)

In [90]:
#batchsize 90 on the A100
#batchsize 40 on the L4
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
)


In [91]:
trainer.train()

Step,Training Loss
500,1.559200
1000,1.415200
1500,1.330700


TrainOutput(global_step=1530, training_loss=1.4326687457514744, metrics={'train_runtime': 844.6563, 'train_samples_per_second': 72.171, 'train_steps_per_second': 1.811, 'total_flos': 1945545335635968.0, 'train_loss': 1.4326687457514744, 'epoch': 10.0})

In [95]:
model.save_pretrained("./shakespeare-scratch")

In [58]:
# prompt: use extract n grams functions to find al bigram in corput, iterate through them, inputting them to the model. calculate the probabilities of the last token logits, and the shannon entropy. Then average for all bigrams
def extract_unique_n_minus_1_grams(corpus, n):
    n_minus_1_grams = set()
    for sentence in corpus:
        sentence = ''.join(sentence)
        for i in range(len(sentence) - n + 2):
            context = sentence[i:i + n - 1]
            if context == 'xz':
              print(sentence)
            n_minus_1_grams.add(context)
    return n_minus_1_grams


def calculate_bigram_probabilities(model, tokenizer, corpus):

    bigrams = extract_unique_n_minus_1_grams(corpus, 3)
    probabilities = []
    entropies = []

    for bigram in bigrams:
        input_ids = torch.tensor(tokenizer.encode(bigram)).unsqueeze(0)
        input_ids = input_ids.to(model.device)

        outputs = model(input_ids, labels=input_ids)
        loss, logits = outputs[:2]

        last_token_logits = logits[0, -1, :]

        # Apply softmax to convert logits to probabilities
        probs = torch.softmax(last_token_logits, dim=-1)

        # Calculate Shannon entropy
        entropy = -torch.sum(probs * torch.log2(probs))

        probabilities.append(probs)
        entropies.append(entropy)

    return bigrams, probabilities, entropies


bigrams, probabilities, entropies = calculate_bigram_probabilities(model, tokenizer, joined_sents)

average_probability = sum(probs.mean() for probs in probabilities) / len(probabilities)
average_entropy = sum(entropy for entropy in entropies) / len(entropies)

print("Average bigram probability:", average_probability)
print("Average bigram entropy:", average_entropy)


Average bigram probability: tensor(0.0357, device='cuda:0', grad_fn=<DivBackward0>)
Average bigram entropy: tensor(2.5939, device='cuda:0', grad_fn=<DivBackward0>)


In [133]:
input_ids = torch.tensor(tokenizer.encode('th')).unsqueeze(0)
input_ids = input_ids.to(model.device)
outputs = model(input_ids, labels=input_ids)

loss, logits = outputs[:2]

last_token_logits = logits[0, -1, :]


probs = torch.softmax(last_token_logits, dim=-1)

num_preds=3
# Get the top num_preds predicted characters and their probabilities
top_preds = torch.topk(probs, num_preds)
predicted_chars = []
predicted_probs = []
predicted_ids = []
for i in range(num_preds):
    pred_id = top_preds.indices[i].item()
    pred_char = tokenizer.decode([pred_id])
    pred_prob = top_preds.values[i].item()

    predicted_ids.append(pred_id)
    predicted_chars.append(pred_char)
    predicted_probs.append(pred_prob)

entropy = -torch.sum(probs * torch.log2(probs))

print(predicted_chars, predicted_ids, predicted_probs, entropy)

['i', 'e', 'y'] [3, 16, 9] [0.49341443181037903, 0.4308587908744812, 0.04180855676531792] tensor(1.4910, device='cuda:0', grad_fn=<NegBackward0>)


In [104]:
full_test = ''.join(flat_corpus_v)
full_train = ''.join(flat_corpus_t)

In [142]:

def extract_unique_n_minus_1_grams(corpus, n):
    n_minus_1_grams = set()
    for sentence in corpus:
        sentence = ''.join(sentence)
        for i in range(len(sentence) - n + 2):
            context = sentence[i:i + n - 1]
            if context == 'xz':
              print(sentence)
            n_minus_1_grams.add(context)
    return n_minus_1_grams

def find_probabilities(unique_strings, long_string):
    frequencies = {string: 0 for string in unique_strings}

    # Length of the unique strings
    string_length = len(list(unique_strings)[0])

    # Total number of possible substrings of the same length in the long string
    total_substrings = len(long_string) - string_length + 1

    for string in unique_strings:
        frequencies[string] = long_string.count(string)


    probabilities = {string: freq / total_substrings for string, freq in frequencies.items()}

    return probabilities


def calculate_bigram_probabilities(model, tokenizer, corpus, test_set, ngram = 3):

    bigrams = extract_unique_n_minus_1_grams(corpus, ngram)
    weights = find_probabilities(bigrams, test_set)

    probabilities = []
    entropies = []

    for bigram in bigrams:
        input_ids = torch.tensor(tokenizer.encode(bigram)).unsqueeze(0)
        input_ids = input_ids.to(model.device)

        outputs = model(input_ids, labels=input_ids)
        loss, logits = outputs[:2]

        last_token_logits = logits[0, -1, :]

        # Apply softmax to convert logits to probabilities
        probs = torch.softmax(last_token_logits, dim=-1)

        # Calculate Shannon entropy
        entropy = torch.sum(probs * torch.log2(1/probs))
        entropy = entropy * weights[bigram]

        probabilities.append(probs)
        entropies.append(entropy)

    return bigrams, probabilities, entropies


bigrams, probabilities, entropies = calculate_bigram_probabilities(model, tokenizer, joined_sents, full_test)

average_probability = sum(probs.mean() for probs in probabilities) / len(probabilities)
weighted_entropy = sum(entropy for entropy in entropies)

print("Average bigram probability:", average_probability)
print("Bigram entropy:", weighted_entropy)


Average bigram probability: tensor(0.0357, device='cuda:0', grad_fn=<DivBackward0>)
Bigram entropy: tensor(2.3457, device='cuda:0', grad_fn=<AddBackward0>)
